In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sqlite3

In [2]:
db_conn = sqlite3.connect("data/us_sales.db")
## don't need cursor if you're not writing to table

In [3]:
## Subset looking at Midwest Sales

In [18]:
df = pd.read_sql_query(
con = db_conn,
    sql = """select 
                s.OrderDate,
                r.Region,
                sum(s.OrderQuantity) cnt
    
             from sales s
             
             join store_locations sl
             on s.storeid = sl.storeid
             
             join regions r
             on sl.StateCode = r.StateCode
             
             where Region = 'Midwest'
             group by OrderDate, Region
    
    """

)

df['OrderDate'] = pd.to_datetime(df['OrderDate'])

In [ ]:
## Run when done with connection
#db_conn.close()

In [19]:
df.head()

,OrderDate,Region,cnt
0,2018-05-31,Midwest,9
1,2018-06-01,Midwest,9
2,2018-06-02,Midwest,15
3,2018-06-03,Midwest,15
4,2018-06-06,Midwest,8


In [6]:
#check the shape of data
df.shape

(790, 3)

In [7]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   OrderDate  790 non-null    datetime64[ns]
 1   Region     790 non-null    object        
 2   cnt        790 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 18.6+ KB


In [20]:
df.head()

,OrderDate,Region,cnt
0,2018-05-31,Midwest,9
1,2018-06-01,Midwest,9
2,2018-06-02,Midwest,15
3,2018-06-03,Midwest,15
4,2018-06-06,Midwest,8


In [21]:
# create moving-averages
df['MA60'] = df['cnt'].rolling(60).mean()
df['MA365'] = df['cnt'].rolling(365).mean()
# plot 
import plotly.express as px
fig = px.line(df, x="OrderDate", y=['cnt', 'MA60', 'MA365'], title='US Sales - Midwest', template = 'presentation')
fig.show()

In [22]:
# drop moving-average columns
df.drop(['MA60', 'MA365'], axis=1, inplace=True)
# set timestamp to index
df.set_index('OrderDate', drop=True, inplace=True)
#resample timeseries to hourly 
#df = df.resample('H').sum()
# creature features from date
df['day'] = [i.day for i in df.index]
df['day_name'] = [i.day_name() for i in df.index]
df['day_of_year'] = [i.dayofyear for i in df.index]
df['week_of_year'] = [i.weekofyear for i in df.index]
df['is_weekday'] = [i.isoweekday() for i in df.index]
df.head()

,Region,cnt,day,day_name,day_of_year,week_of_year,is_weekday
OrderDate,,,,,,,
2018-05-31,Midwest,9,31,Thursday,151,22,4
2018-06-01,Midwest,9,1,Friday,152,22,5
2018-06-02,Midwest,15,2,Saturday,153,22,6
2018-06-03,Midwest,15,3,Sunday,154,22,7
2018-06-06,Midwest,8,6,Wednesday,157,23,3


In [23]:
# init setup
from pycaret.anomaly import *
s = setup(df, session_id = 123)

,Description,Value
0,session_id,123
1,Original Data,"(790, 7)"
2,Missing Values,False
3,Numeric Features,4
4,Categorical Features,3
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(790, 19)"
9,CPU Jobs,-1


In [24]:
# train model
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

,Region,cnt,day,day_name,day_of_year,week_of_year,is_weekday,Anomaly,Anomaly_Score
OrderDate,,,,,,,,,
2018-05-31,Midwest,9,31,Thursday,151,22,4,0,-0.018717
2018-06-01,Midwest,9,1,Friday,152,22,5,0,-0.046081
2018-06-02,Midwest,15,2,Saturday,153,22,6,0,-0.028582
2018-06-03,Midwest,15,3,Sunday,154,22,7,0,-0.023789
2018-06-06,Midwest,8,6,Wednesday,157,23,3,0,-0.046446


In [25]:
iforest_results.to_csv (r'export_dataframe.csv', index = False, header=True)
print (iforest_results)

             Region  cnt  day   day_name  day_of_year  week_of_year  \
OrderDate                                                             
2018-05-31  Midwest    9   31   Thursday          151            22   
2018-06-01  Midwest    9    1     Friday          152            22   
2018-06-02  Midwest   15    2   Saturday          153            22   
2018-06-03  Midwest   15    3     Sunday          154            22   
2018-06-06  Midwest    8    6  Wednesday          157            23   
...             ...  ...  ...        ...          ...           ...   
2020-12-25  Midwest    8   25     Friday          360            52   
2020-12-26  Midwest   20   26   Saturday          361            52   
2020-12-27  Midwest   15   27     Sunday          362            52   
2020-12-28  Midwest    2   28     Monday          363            53   
2020-12-29  Midwest    1   29    Tuesday          364            53   

            is_weekday  Anomaly  Anomaly_Score  
OrderDate                  

In [26]:
iforest_anomaly=iforest_results[iforest_results['Anomaly']==1]
iforest_anomaly.shape

(79, 9)

In [27]:
# save a model
save_model(iforest, 'iforest_20210925')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  IFo

In [46]:
plot_model(iforest)

In [28]:
# check anomalies
iforest_results[iforest_results['Anomaly'] == 1].head()

,Region,cnt,day,day_name,day_of_year,week_of_year,is_weekday,Anomaly,Anomaly_Score
OrderDate,,,,,,,,,
2018-08-17,Midwest,30,17,Friday,229,33,5,1,0.005126
2018-11-01,Midwest,31,1,Thursday,305,44,4,1,0.049228
2018-11-27,Midwest,1,27,Tuesday,331,48,2,1,0.005272
2018-12-04,Midwest,2,4,Tuesday,338,49,2,1,0.000174
2018-12-20,Midwest,33,20,Thursday,354,51,4,1,0.044652


In [29]:
import plotly.graph_objects as go
# plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y="cnt", title='US Sales Anomaly Detection - Midwest', template = 'presentation')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['cnt'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=5)))
        
fig.show()